# 5gdeploy - 2-slice with 2 UPFs

This notebook demonstrates how to install [5gdeploy](https://github.com/usnistgov/5gdeploy) on FABRIC nodes and run the `5gdeploy/scenario/20231017` scenario.

## Step 0: Import FABlib

This notebook requires a project with these permissions: VM.NoLimit.

In [ ]:
import json
import random
import shlex
import tabulate
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
fablib.show_config()

## Step 1: Create Slice

This step creates a slice with four nodes:

* **cp** runs the control plane and RAN simulators.
* **dn** runs Data Network containers and associated traffic generators.
* **u1** runs UPF1.
* **u4** runs UPF4.

The **cp** node also serves the primary host that controls the experiment.

The slice has three networks:

* **FABNetv4** is for SSH control and VXLAN tunnels.
* **N3** carries user traffic between gNBs and UPFs.
* **N6** carries user traffic between UPFs and Data Networks.

In [ ]:
# modify these parameters if necessary

# slice name
SLICE_NAME = '5gdeploy-20231017'
# FABRIC site name - IPv6 management address is OK, this notebook is compatible with DNS64
SITE = 'INDI'

# CPU cores reserved for operating system, minimum is 2
CORES_OS = 2
# CPU cores reserved for 5G network functions, minimum is 6
CORES_NF = 6
# CPU cores reserved for traffic generators, minimum is 4
CORES_TG = 8

# no need to change anything below
assert CORES_OS >= 2
assert CORES_NF >= 6
assert CORES_TG >= 4
cpuset_os = f'0-{CORES_OS-1}'
cpuset_nf = f'{CORES_OS}-{CORES_OS+CORES_NF-1}'
cpuset_tg = f'{CORES_OS+CORES_NF}-{CORES_OS+CORES_NF+CORES_TG-1}'

In [ ]:
# create new slice
slice = fablib.new_slice(name=SLICE_NAME)

# define nodes
kwargs = {'site':SITE,'ram':16,'disk':100,'image':'docker_ubuntu_24'}
node_cp = slice.add_node(name='cp', **kwargs, cores=CORES_OS+CORES_NF+CORES_TG)
node_dn = slice.add_node(name='dn', **kwargs, cores=CORES_OS+CORES_NF+CORES_TG)
node_u1 = slice.add_node(name='u1', **kwargs, cores=CORES_OS+CORES_NF)
node_u4 = slice.add_node(name='u4', **kwargs, cores=CORES_OS+CORES_NF)

# define control network
node_cp.add_fabnet()
node_dn.add_fabnet()
node_u1.add_fabnet()
node_u4.add_fabnet()

# define N3 network
intfs_n3 = []
intfs_n3 += node_cp.add_component(model='NIC_Basic', name='n3g0').get_interfaces()
intfs_n3 += node_cp.add_component(model='NIC_Basic', name='n3g1').get_interfaces()
intfs_n3 += node_u1.add_component(model='NIC_Basic', name='n3u1').get_interfaces()
intfs_n3 += node_u4.add_component(model='NIC_Basic', name='n3u4').get_interfaces()
net_n3 = slice.add_l2network(name='N3', interfaces=intfs_n3)

# define N6 network
intfs_n6 = []
intfs_n6 += node_dn.add_component(model='NIC_Basic', name='n6d0').get_interfaces()
intfs_n6 += node_dn.add_component(model='NIC_Basic', name='n6d1').get_interfaces()
intfs_n6 += node_dn.add_component(model='NIC_Basic', name='n6d2').get_interfaces()
intfs_n6 += node_u1.add_component(model='NIC_Basic', name='n6u1').get_interfaces()
intfs_n6 += node_u4.add_component(model='NIC_Basic', name='n6u4').get_interfaces()
net_n6 = slice.add_l2network(name='N6', interfaces=intfs_n6)

# submit slice request
slice.submit()

## Step 2: Install Dependencies

This step performs initial configuration on every node:

1. Install Node.js and other dependencies.
2. Configure CPU isolation in systemd.
3. Reboot the node.

In [ ]:
# retrieve the slice
slice = fablib.get_slice(name=SLICE_NAME)

# perform initial configuration and install dependencies, then reboot
execute_threads = {}
for node in slice.get_nodes():
    execute_threads[node] = node.execute_thread(f'''
        sudo hostnamectl set-hostname {node.get_name()}
        echo 'set enable-bracketed-paste off' | sudo tee -a /etc/inputrc
        curl -fsLS https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg
        echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_22.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list
        sudo apt update
        sudo DEBIAN_FRONTEND=noninteractive apt full-upgrade -y
        sudo DEBIAN_FRONTEND=noninteractive apt purge -y nano
        echo 'wireshark-common wireshark-common/install-setuid boolean true' | sudo debconf-set-selections
        sudo DEBIAN_FRONTEND=noninteractive apt install -y clang-format-15 httpie linux-generic moreutils nodejs python3-libconf wireshark-common
        sudo adduser $(id -un) wireshark
        sudo snap install yq
        echo | sudo mkdir -p /etc/systemd/system/init.scope.d /etc/systemd/system/service.d /etc/systemd/system/user.slice.d /etc/systemd/system/docker-.scope.d
        echo -e "[Scope]\nAllowedCPUs={cpuset_os}" | sudo tee /etc/systemd/system/init.scope.d/cpuset.conf
        echo -e "[Service]\nAllowedCPUs={cpuset_os}" | sudo tee /etc/systemd/system/service.d/cpuset.conf
        echo -e "[Slice]\nAllowedCPUs={cpuset_os}" | sudo tee /etc/systemd/system/user.slice.d/cpuset.conf
        echo -e "[Scope]\nAllowedCPUs={CORES_OS}-{node.get_cores()-1}" | sudo tee /etc/systemd/system/docker-.scope.d/cpuset.conf
        sudo reboot
    ''')
for thread in execute_threads.values():
    thread.result()
slice.wait_ssh(progress=True)
slice.post_boot_config()

## Step 3: Install 5gdeploy

This step downloads 5gdeploy on the primary host and invokes its install script.

In [ ]:
# retrieve primary host node
slice = fablib.get_slice(name=SLICE_NAME)
node_cp = slice.get_node(name='cp')

In [ ]:
# clone 5gdeploy repository
stdout, stderr = node_cp.execute('''
    rm -rf ~/5gdeploy
    git clone https://github.com/usnistgov/5gdeploy.git
''')

In [ ]:
# run install script
stdout, stderr = node_cp.execute('~/5gdeploy/install.sh')

## Step 4: Prepare for Multi-Host Deployment

This step gathers information about network interfaces, and then creates an SSH key for the primary host to control secondary hosts.

In [ ]:
# retrieve slice and nodes
slice = fablib.get_slice(name=SLICE_NAME)
node_cp = slice.get_node(name='cp')
node_dn = slice.get_node(name='dn')
node_u1 = slice.get_node(name='u1')
node_u4 = slice.get_node(name='u4')

# retrieve control IP addresses
net_fabnet = f'FABNET_IPv4_{SITE}'
ctrl_cp = node_cp.get_interface(network_name=net_fabnet).get_ip_addr()
ctrl_dn = node_dn.get_interface(network_name=net_fabnet).get_ip_addr()
ctrl_u1 = node_u1.get_interface(network_name=net_fabnet).get_ip_addr()
ctrl_u4 = node_u4.get_interface(network_name=net_fabnet).get_ip_addr()

# retrieve N3 MAC addresses
n3_g0 = node_cp.get_component('n3g0').get_interfaces()[0].get_mac()
n3_g1 = node_cp.get_component('n3g1').get_interfaces()[0].get_mac()
n3_u1 = node_u1.get_component('n3u1').get_interfaces()[0].get_mac()
n3_u4 = node_u4.get_component('n3u4').get_interfaces()[0].get_mac()
n3_2gnb = ','.join([n3_g0, n3_g1])

# retrieve N6 MAC addresses
n6_d0 = node_dn.get_component('n6d0').get_interfaces()[0].get_mac()
n6_d1 = node_dn.get_component('n6d1').get_interfaces()[0].get_mac()
n6_d2 = node_dn.get_component('n6d2').get_interfaces()[0].get_mac()
n6_u1 = node_u1.get_component('n6u1').get_interfaces()[0].get_mac()
n6_u4 = node_u4.get_component('n6u4').get_interfaces()[0].get_mac()
n6_3dn = ','.join([n6_d0, n6_d1, n6_d2])

tabulate.tabulate([
    ['cp', node_cp.get_management_ip(), ctrl_cp, n3_2gnb, ''],
    ['dn', node_dn.get_management_ip(), ctrl_dn, '', n6_3dn],
    ['u1', node_u1.get_management_ip(), ctrl_u1, n3_u1, n6_u1],
    ['u4', node_u4.get_management_ip(), ctrl_u4, n3_u4, n6_u4],
], headers=['node', 'management', 'ctrl', 'N3', 'N6'], tablefmt='html')

In [ ]:
# create SSH key pair on primary host
stdout, stderr = node_cp.execute('''
    rm -f ~/.ssh/id_ed25519 ~/.ssh/id_ed25519.pub
    ssh-keygen -f ~/.ssh/id_ed25519 -N '' -C 5gdeploy -t ed25519
''')
pubkey, stderr = node_cp.execute('cat ~/.ssh/id_ed25519.pub')

# grant permission on secondary hosts
for node in slice.get_nodes():
    stdout, stderr = node.execute(f'''
        sed -i '/^ssh-ed25519.*5gdeploy$/ d' ~/.ssh/authorized_keys
        echo {shlex.quote(pubkey)} >>~/.ssh/authorized_keys
    ''')

# save host keys
stdout, stderr = node_cp.execute(f'''
    rm -f ~/.ssh/known_hosts
    ssh-keyscan {ctrl_cp} {ctrl_dn} {ctrl_u1} {ctrl_u4} | tee ~/.ssh/known_hosts
''')

## Step 5: Generate Scenario

This step invokes `5gdeploy/scenario/generate.sh` script to generate a predefined scenario template into netdef.json and then Docker Compose context.

Modify the topology parameters first and run the parameters block.

After that, there are multiple blocks provided for different 5G core and RAN software.
You should choose one option and run only that block.
You can switch between options by running another block, which would automatically stop the running scenario.

In [ ]:
# assign topology parameters
N_GNBS = 2
N_PHONES = 1
N_VEHICLES = 1

# no need to change anything below
assert N_GNBS <= 2
nUes = N_PHONES + N_VEHICLES
nPduSessions = N_PHONES + 2 * N_VEHICLES

### free5GC + PacketRusher

PacketRusher supports exactly one UE per gNB.
Therefore, gNB quantity must equal the sum of phone quantity and vehicle quantity.

In [ ]:
IMPLS = ('free5gc', 'free5gc', 'packetrusher')
assert N_GNBS == nUes
nPduSessions = nUes

# prepare generate.sh command line
generate_cmd = f'''
./generate.sh 20231017
  +gnbs={N_GNBS} +phones={N_PHONES} +vehicles={N_VEHICLES}
  --cp=free5gc --up=free5gc --ran=packetrusher
  --dn-workers=0
  --bridge='mgmt | vx | {ctrl_cp},{ctrl_dn}'
  --bridge='n4 | vx | {ctrl_cp},{ctrl_u1},{ctrl_u4}'
  --bridge='n3 | eth | gnb*={n3_2gnb} upf1={n3_u1} upf4={n3_u4}'
  --bridge='n6 | eth | dn_*={n6_3dn} upf1={n6_u1} upf4={n6_u4}'
  --place='dn_*@{ctrl_dn}({cpuset_nf})'
  --place='upf1@{ctrl_u1}({cpuset_nf})'
  --place='upf4@{ctrl_u4}({cpuset_nf})'
  --place='*@({cpuset_nf})'
'''
print(generate_cmd)

# invoke generate.sh
stdout, stderr = node_cp.execute(f'''
    cd ~/5gdeploy/scenario
    {generate_cmd.replace(chr(10), ' ')}
''')

### Open5GS + eUPF + PacketRusher

PacketRusher supports exactly one UE per gNB.
Therefore, gNB quantity must equal the sum of phone quantity and vehicle quantity.

In [ ]:
IMPLS = ('open5gs', 'eupf', 'packetrusher')
assert N_GNBS == nUes
nPduSessions = nUes

# prepare generate.sh command line
generate_cmd = f'''
./generate.sh 20231017
  +gnbs={N_GNBS} +phones={N_PHONES} +vehicles={N_VEHICLES}
  --cp=open5gs --up=eupf --ran=packetrusher
  --dn-workers=0
  --bridge='mgmt | vx | {ctrl_cp},{ctrl_dn},{ctrl_u1},{ctrl_u4}'
  --bridge='n4 | vx | {ctrl_cp},{ctrl_u1},{ctrl_u4}'
  --bridge='n3 | eth | gnb*={n3_2gnb} upf1={n3_u1} upf4={n3_u4}'
  --bridge='n6 | eth | dn_*={n6_3dn} upf1={n6_u1} upf4={n6_u4}'
  --place='dn_*@{ctrl_dn}({cpuset_nf})'
  --place='upf1@{ctrl_u1}({cpuset_nf})'
  --place='upf4@{ctrl_u4}({cpuset_nf})'
  --place='*@({cpuset_nf})'
'''
print(generate_cmd)

# invoke generate.sh
stdout, stderr = node_cp.execute(f'''
    cd ~/5gdeploy/scenario
    {generate_cmd.replace(chr(10), ' ')}
''')

### Open5GS + UERANSIM

UERANSIM is mainly intended for protocol verification.
Its dataplane performance is very low.
If you use traffic generators, please limit traffic volume to be no more than 10 Mbps.

In [ ]:
IMPLS = ('open5gs', 'open5gs', 'ueransim')

# prepare generate.sh command line
generate_cmd = f'''
./generate.sh 20231017
  +gnbs={N_GNBS} +phones={N_PHONES} +vehicles={N_VEHICLES}
  --cp=open5gs --up=open5gs --ran=ueransim
  --dn-workers=0
  --bridge='mgmt | vx | {ctrl_cp},{ctrl_dn}'
  --bridge='n4 | vx | {ctrl_cp},{ctrl_u1},{ctrl_u4}'
  --bridge='n3 | eth | gnb*={n3_2gnb} upf1={n3_u1} upf4={n3_u4}'
  --bridge='n6 | eth | dn_*={n6_3dn} upf1={n6_u1} upf4={n6_u4}'
  --place='dn_*@{ctrl_dn}({cpuset_nf})'
  --place='upf1@{ctrl_u1}({cpuset_nf})'
  --place='upf4@{ctrl_u4}({cpuset_nf})'
  --place='*@({cpuset_nf})'
'''
print(generate_cmd)

# invoke generate.sh
stdout, stderr = node_cp.execute(f'''
    cd ~/5gdeploy/scenario
    {generate_cmd.replace(chr(10), ' ')}
''')

### free5GC + NDN-DPDK/eUPF + PacketRusher

**UPF1** (serves `internet` Data Network) runs [NDN-DPDK](https://github.com/usnistgov/ndn-dpdk) forwarder along with `ndndpdk-upf` PFCP agent.
This implementation supports both IPv4 and NDN traffic.

**UPF4** (serves `vcam` and `vctl` Data Networks) runs eUPF.

PacketRusher supports exactly one UE per gNB.
Therefore, gNB quantity must equal the sum of phone quantity and vehicle quantity.

In [ ]:
IMPLS = ('free5gc', 'ndndpdk+eupf', 'packetrusher')
assert N_GNBS == nUes
nPduSessions = nUes

# pull NDN-DPDK Docker images and setup hugepages
stdout, stderr = node_u1.execute(f'''
    docker rm -f upf1 upfsvc1
    if ! [[ -f /usr/local/bin/dpdk-hugepages.py ]]; then
        docker pull ghcr.io/usnistgov/ndn-dpdk
        docker tag ghcr.io/usnistgov/ndn-dpdk localhost/ndn-dpdk
        docker run --rm localhost/ndn-dpdk sh -c 'tar -c /usr/local/bin/dpdk-*.py /usr/local/share/ndn-dpdk' | sudo tar -x -C /
    fi
    sudo sh -c 'while ! dpdk-hugepages.py --setup 4G; do sleep 1; done'
    dpdk-hugepages.py --show
''')

# determine N3 and N6 Ethernet adapter PCI addresses
n3PciAddr, stderr = node_u1.execute(f'''
    IFNAME=$(ip -j link | jq -r --arg MAC {n3_u1.lower()} '.[] | select(.address==$MAC) | .ifname' | head -1)
    basename $(readlink -f /sys/class/net/$IFNAME/device)
''')
n3PciAddr = n3PciAddr.strip()

# prepare NDN-DPDK activation parameters
assert CORES_NF >= 4
upf1_activate = {
    'eal': {
        'lcoreMain': CORES_OS,
    },
    'lcoreAlloc': {
        'RX': [CORES_OS+1],
        'TX': [CORES_OS+2],
        'FWD': [CORES_OS+3],
    },
    '5gdeploy-create-eth-port': f'--pci {n3PciAddr}'
}

# prepare generate.sh command line
generate_cmd = f'''
./generate.sh 20231017
  +gnbs={N_GNBS} +phones={N_PHONES} +vehicles={N_VEHICLES}
  --cp=free5gc --up=upf1=ndndpdk --up=eupf --ran=packetrusher
  --ndndpdk-gtpip=true --ndndpdk-activate=$HOME/ndndpdk-activate
  --dn-workers=0
  --bridge='mgmt | vx | {ctrl_cp},{ctrl_dn},{ctrl_u1},{ctrl_u4}'
  --bridge='n4 | vx | {ctrl_cp},{ctrl_u1},{ctrl_u4}'
  --bridge='n3 | eth | gnb*={n3_2gnb} upf1={n3_u1} upf4={n3_u4}'
  --bridge='n6 | eth | dn_*={n6_3dn} upf1={n6_u1} upf4={n6_u4}'
  --place='dn_*@{ctrl_dn}({cpuset_nf})'
  --place='upf1@{ctrl_u1}({CORES_OS})'
  --place='upfsvc1@{ctrl_u1}'
  --place='upf4@{ctrl_u4}({cpuset_nf})'
  --place='*@({cpuset_nf})'
'''
print(generate_cmd)

# save NDN-DPDK activation parameters and invoke generate.sh
stdout, stderr = node_cp.execute(f'''
    mkdir -p ~/ndndpdk-activate
    echo {shlex.quote(json.dumps(upf1_activate, indent=2))} | tee ~/ndndpdk-activate/upf1.json
    cd ~/5gdeploy/scenario
    {generate_cmd.replace(chr(10), ' ')}
''')


## Step 6: Launch Scenario

This step invokes `compose.sh` script to launch the scenario in Docker Compose, and then checks its status.
You can run the `ps` and `list-pdu` subcommands repeatedly.
Once the PDU sessions are established, you can proceed to the next step for traffic generators.

In [ ]:
# show network function placement and cpuset
stdout, stderr = node_cp.execute('''
    cd ~/compose/20231017
    ~/5gdeploy/compose/place-report.sh compose.yml
''')

In [ ]:
# launch the scenario
stdout, stderr = node_cp.execute('~/compose/20231017/compose.sh up')

In [ ]:
# check scenario status
stdout, stderr = node_cp.execute('~/compose/20231017/compose.sh ps')

In [ ]:
# list PDU sessions
stdout, stderr = node_cp.execute('~/compose/20231017/compose.sh list-pdu')

## Step 7: Run Traffic Generators

This step runs traffic generators between UEs and Data Networks.

There are multiple blocks provided for different traffic generators.
You can run any of them in any order.

Traffic generators using `tgcs` subcommand will deposit detailed results in `~/compose/20231017/tg` directory.
You can SSH into the primary host and view these files.

### nmap

Nmap scans the UE subnet to find which IP addresses are online.
You can use this command to confirm PDU session reachability.

In [ ]:
stdout, stderr = node_cp.execute('~/compose/20231017/compose.sh nmap')

### iPerf2

iPerf2 performs a network throughput and latency benchmark.
You can adjust the parameters near the top of this code block.

In [ ]:
# traffic direction, either 'UL' or 'DL'
DIR = 'DL'
# traffic duration in seconds
DURATION = 30
# per-flow bandwidth in Mbps
BANDWIDTH = 400
# number of flows
FLOWS = 2

# no need to change anything below
assert DIR in ['UL', 'DL']
assert DURATION >= 10
assert BANDWIDTH >= 1
assert FLOWS >= 1
assert CORES_TG >= FLOWS * nPduSessions

# prepare tgcs command line
tgcs_cmd = f'''
./compose.sh tgcs
  --place='*@({cpuset_tg})' --place='*@{ctrl_dn}({cpuset_tg})' --wait-timeout={60+DURATION}
  --iperf2='* | *
    | #cpus={FLOWS} {'#R' if DIR=='DL' else ''} -t {DURATION} -i 0.1 -u -l 1250 -b {BANDWIDTH}m --trip-times -P {FLOWS}
    | #cpus={FLOWS} -u -l 1250 -i 0.1'
'''
print(tgcs_cmd)

# invoke traffic generator
stdout, stderr = node_cp.execute(f'''
    cd ~/compose/20231017
    {tgcs_cmd.replace(chr(10), ' ')}
    ./tg.sh upload
    ./tg.sh
''')

### NFD + ndnping

This block deploys NDN Forwarding Daemon (NFD) on each *phone* over the `internet` DNN, and then runs ndnping to verify reachability.
You can adjust the parameters near the top of this code block.

If you are using NDN-DPDK as UPF1, run the blocks marked NDN-DPDK UPF only.
Otherwise, skip these blocks.
If you defined two *phones* in the topology (`N_GNBS, N_PHONES, N_VEHICLES = 2, 2, 0`), you would see Interest aggregation benefits through the counters.

In [ ]:
# interval in milliseconds
INTERVAL = 100
# starting sequence number
STARTSEQ = random.randint(0,0x100000000)
# packet quantity
PKTCOUNT = 1000
# reply payload length
PAYLOADLEN = 1000

# no need to change anything below
assert INTERVAL >= 10
assert PKTCOUNT >= 10
assert PAYLOADLEN >= 0 and PAYLOADLEN <= 8000

In [ ]:
# run this block only if UPF1 is NDN-DPDK
assert 'ndndpdk' in IMPLS[1]

# create face toward producer in NDN-DPDK forwarder
stdout, stderr = node_u1.execute(f'''
    exec 5>&1
    if [[ -z $(docker exec upf1 ndndpdk-ctrl list-ethdev |
               jq -c --arg MAC {n6_u1.lower()} 'select(.macAddr==$MAC)' |
               tee /dev/fd/5) ]]; then
        docker exec upf1 ndndpdk-ctrl create-eth-port --netif n6 --xdp
    fi
    FACEID=$(docker exec upf1 ndndpdk-ctrl list-faces |
             jq -c --arg MAC {n6_d0.lower()} 'select(.locator.remote==$MAC)' |
             tee /dev/fd/5 | jq -r '.id')
    if [[ -z $FACEID ]]; then
        FACEID=$(docker exec upf1 ndndpdk-ctrl create-ether-face --local {n6_u1.lower()} --remote {n6_d0.lower()} |
                 tee /dev/fd/5 | jq -r '.id')
    fi
    docker exec upf1 ndndpdk-ctrl insert-fib --name / --nh $FACEID
''')

# save NDN-DPDK face counters before traffic
stdout, stderr = node_u1.execute('docker exec upf1 ndndpdk-ctrl list-faces -cnt')
faceCounters0 = dict([ (x['id'], x) for x in [ json.loads(line) for line in stdout.strip().split('\n') ] ])

In [ ]:
# always run this block

# prepare tgcs command line
tgcs_cmd = f'''
./compose.sh tgcs
  --place='*@({cpuset_tg})' --place='*@{ctrl_dn}({cpuset_tg})' --wait-timeout={60+int(INTERVAL*PKTCOUNT/1000)}
  --ndnping='internet | * | -i {INTERVAL} -n {STARTSEQ} -c {PKTCOUNT} | -s {PAYLOADLEN}'
'''
print(tgcs_cmd)

# launch NFD and invoke traffic generator
stdout, stderr = node_cp.execute(f'''
    cd ~/compose/20231017
    ./compose.sh nfd --dnn=internet
    {tgcs_cmd.replace(chr(10), ' ')}
    ./tg.sh
''')

In [ ]:
# run this block only if UPF1 is NDN-DPDK
assert 'ndndpdk' in IMPLS[1]

# save NDN-DPDK face counters after traffic
stdout, stderr = node_u1.execute('docker exec upf1 ndndpdk-ctrl list-faces -cnt')
faceCounters1 = dict([ (x['id'], x) for x in [ json.loads(line) for line in stdout.strip().split('\n') ] ])

# show traffic counter increments
METRICS = ['rxInterests', 'rxData', 'rxNacks', 'txInterests', 'txData', 'txNacks']
rows = []
for id, face1 in faceCounters1.items():
    try:
        face0 = faceCounters0[id]
    except:
        continue
    loc, cnt0, cnt1 = face1['locator'], face0['counters'], face1['counters']
    row = [id, loc['scheme'], loc.get('innerRemoteIP')]
    for metric in METRICS:
        row.append(cnt1[metric] - cnt0[metric])
    rows.append(row)
tabulate.tabulate(rows, headers=['id', 'scheme', 'UE IP'] + METRICS, tablefmt='html')

## Step 8: Cleanup

You can stop the scenario and later re-launch it.
Once you are done for the day, please delete the slice, which would destroy the virtual machines and release FABRIC resources.

In [ ]:
# stop the scenario
stdout, stderr = node_cp.execute('~/compose/20231017/compose.sh down')

In [ ]:
# delete the slice
slice = fablib.get_slice(SLICE_NAME)
slice.delete()